# Importing Libraries

In [1]:
%matplotlib notebook
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, utils
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split

Data was scrapped using scrapping tool

# READING THE DATA 

In [2]:
matches=pd.read_csv('./data/PSL_2016-2020.csv')
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 15 columns):
Team_1              127 non-null object
Team_2              127 non-null object
Team1_Code          127 non-null object
Team2_Code          127 non-null object
Season              127 non-null int64
venue               127 non-null object
id                  127 non-null int64
Win_By_Wkts         127 non-null int64
Win_By_Runs         127 non-null int64
toss_winner         127 non-null object
toss_winner_code    127 non-null object
toss_decision       127 non-null object
result              127 non-null object
winner              127 non-null object
winner_code         127 non-null object
dtypes: int64(4), object(11)
memory usage: 15.0+ KB


#### NO NULL VALUES FOUND

# DATA

In [3]:
matches.head()

,Team_1,Team_2,Team1_Code,Team2_Code,Season,venue,id,Win_By_Wkts,Win_By_Runs,toss_winner,toss_winner_code,toss_decision,result,winner,winner_code
0,ISLAMABAD UNITED,QUETTA GLADIATORS,IU,QG,2016,Dubai,1,8,0,QUETTA GLADIATORS,QG,field,normal,QUETTA GLADIATORS,QG
1,KARACHI KINGS,LAHORE QALANDARS,KK,LQ,2016,Dubai,2,7,0,KARACHI KINGS,KK,field,normal,KARACHI KINGS,KK
2,PESHAWAR ZALMI,ISLAMBAD UNITED,PZ,IU,2016,Dubai,3,0,24,PESHAWAR ZALMI,PZ,bat,normal,PESHAWAR ZALMI,PZ
3,QUETTA GLADIATORS,KARACHI KINGS,QG,KK,2016,Dubai,4,8,0,QUETTA GLADIATORS,QG,field,normal,QUETTA GLADIATORS,QG
4,LAHORE QALANDARS,PESHAWAR ZALMI,LQ,PZ,2016,Dubai,5,9,0,PESHAWAR ZALMI,PZ,field,normal,PESHAWAR ZALMI,PZ


In [4]:
matches.tail()

,Team_1,Team_2,Team1_Code,Team2_Code,Season,venue,id,Win_By_Wkts,Win_By_Runs,toss_winner,toss_winner_code,toss_decision,result,winner,winner_code
122,LAHORE QALANDARS,ISLAMABAD UNITED,LQ,IU,2020,karachi,122,1,0,ISLAMABAD UNITED,IU,field,normal,ISLAMABAD UNITED,IU
123,MULTAN SULTANS,PESHAWAR ZALMI,MS,PZ,2020,karachi,123,6,0,MULTAN SULTANS,MS,field,normal,MULTAN SULTANS,MS
124,ISLAMABAD UNITED,QUETTA GLADIATORS,IU,QG,2020,karachi,124,5,0,QUETTA GLADIATORS,QG,field,normal,QUETTA GLADIATORS,QG
125,MULTAN SULTANS,KARACHI KINGS,MS,KK,2020,karachi,125,0,52,KARACHI KINGS,KK,field,normal,MULTAN SULTANS,MS
126,ISLAMABAD UNITED,QUETTA GLADIATORS,IU,QG,2020,karachi,124,5,0,QUETTA GLADIATORS,QG,field,normal,QUETTA GLADIATORS,QG


In [5]:
encode = {'Team1_Code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6},
          'Team2_Code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6},
          'toss_winner_code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6},
          'winner_code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6,'MT':7,'MA':8},
         'toss_decision': {'field':1,'bat':2}}
matches.replace(encode,inplace=True)
matches.head()

,Team_1,Team_2,Team1_Code,Team2_Code,Season,venue,id,Win_By_Wkts,Win_By_Runs,toss_winner,toss_winner_code,toss_decision,result,winner,winner_code
0,ISLAMABAD UNITED,QUETTA GLADIATORS,4,6,2016,Dubai,1,8,0,QUETTA GLADIATORS,6,1,normal,QUETTA GLADIATORS,6
1,KARACHI KINGS,LAHORE QALANDARS,1,2,2016,Dubai,2,7,0,KARACHI KINGS,1,1,normal,KARACHI KINGS,1
2,PESHAWAR ZALMI,ISLAMBAD UNITED,5,4,2016,Dubai,3,0,24,PESHAWAR ZALMI,5,2,normal,PESHAWAR ZALMI,5
3,QUETTA GLADIATORS,KARACHI KINGS,6,1,2016,Dubai,4,8,0,QUETTA GLADIATORS,6,1,normal,QUETTA GLADIATORS,6
4,LAHORE QALANDARS,PESHAWAR ZALMI,2,5,2016,Dubai,5,9,0,PESHAWAR ZALMI,5,1,normal,PESHAWAR ZALMI,5


In [6]:
matches.describe()

,Team1_Code,Team2_Code,Season,id,Win_By_Wkts,Win_By_Runs,toss_winner_code,toss_decision,winner_code
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,3.543307,3.755906,2017.874016,63.685039,4.212598,8.259843,3.826772,1.149606,4.141732
std,1.641505,1.888632,1.247169,36.415525,5.207793,16.851929,1.869108,0.358098,1.798255
min,1.000000,1.000000,2016.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2017.000000,32.500000,0.000000,0.000000,2.000000,1.000000,3.000000
50%,4.000000,4.000000,2018.000000,64.000000,5.000000,0.000000,4.000000,1.000000,4.000000
75%,5.000000,5.500000,2019.000000,94.500000,7.000000,5.500000,5.000000,1.000000,6.000000
max,6.000000,6.000000,2020.000000,125.000000,50.000000,67.000000,6.000000,2.000000,8.000000


#### For Finding The Team code the key or value

In [7]:
dicVal = encode['winner_code']
print(dicVal['QG']) #key value
print(list(dicVal.keys())[list(dicVal.values()).index(6)]) #find key by value search 

6
QG


In [8]:
df = pd.DataFrame(matches)
df.describe()

,Team1_Code,Team2_Code,Season,id,Win_By_Wkts,Win_By_Runs,toss_winner_code,toss_decision,winner_code
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,3.543307,3.755906,2017.874016,63.685039,4.212598,8.259843,3.826772,1.149606,4.141732
std,1.641505,1.888632,1.247169,36.415525,5.207793,16.851929,1.869108,0.358098,1.798255
min,1.000000,1.000000,2016.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2017.000000,32.500000,0.000000,0.000000,2.000000,1.000000,3.000000
50%,4.000000,4.000000,2018.000000,64.000000,5.000000,0.000000,4.000000,1.000000,4.000000
75%,5.000000,5.500000,2019.000000,94.500000,7.000000,5.500000,5.000000,1.000000,6.000000
max,6.000000,6.000000,2020.000000,125.000000,50.000000,67.000000,6.000000,2.000000,8.000000


# Saving Total Winnig by each team in DataFrame

In [9]:
wins = df["winner_code"]
tot_wins = dict(wins.value_counts())
tot_wins

{6: 30, 5: 28, 4: 27, 1: 18, 3: 10, 2: 10, 8: 2, 7: 2}

In [10]:
df['team1_total_wins'] = df['Team1_Code'].map(tot_wins)
df['team2_total_wins'] = df['Team2_Code'].map(tot_wins)
df.head(3)

,Team_1,Team_2,Team1_Code,Team2_Code,Season,venue,id,Win_By_Wkts,Win_By_Runs,toss_winner,toss_winner_code,toss_decision,result,winner,winner_code,team1_total_wins,team2_total_wins
0,ISLAMABAD UNITED,QUETTA GLADIATORS,4,6,2016,Dubai,1,8,0,QUETTA GLADIATORS,6,1,normal,QUETTA GLADIATORS,6,27,30
1,KARACHI KINGS,LAHORE QALANDARS,1,2,2016,Dubai,2,7,0,KARACHI KINGS,1,1,normal,KARACHI KINGS,1,18,10
2,PESHAWAR ZALMI,ISLAMBAD UNITED,5,4,2016,Dubai,3,0,24,PESHAWAR ZALMI,5,2,normal,PESHAWAR ZALMI,5,28,27


# Setting The Input Parameters

In [11]:
matches = matches[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','winner_code',"team1_total_wins","team2_total_wins"]]
matches.head()

,Team1_Code,Team2_Code,toss_winner_code,toss_decision,winner_code,team1_total_wins,team2_total_wins
0,4,6,6,1,6,27,30
1,1,2,1,1,1,18,10
2,5,4,5,2,5,28,27
3,6,1,6,1,6,30,18
4,2,5,5,1,5,10,28


# Finding out the Total Number of Wins by each Team

In [12]:
temp1=df['toss_winner_code'].value_counts(sort=True)
temp2=df['winner_code'].value_counts(sort=True)
#'No of toss winners by each team'
print('No of toss winners by each team')
for idx, val in temp1.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print('No of match winners by each team')
for idx, val in temp2.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print("Where MA : Match Abandoned\n      MT: Match Tied")

No of toss winners by each team
QG -> 31
PZ -> 28
KK -> 26
IU -> 20
LQ -> 12
MS -> 10
No of match winners by each team
QG -> 30
PZ -> 28
IU -> 27
KK -> 18
MS -> 10
LQ -> 10
MA -> 2
MT -> 2
Where MA : Match Abandoned
      MT: Match Tied


# HISTOGRAM OF TEAM WINS

In [13]:
df['winner_code'].hist(bins=20)

<IPython.core.display.Javascript object>

#### Shows the Team labeled 6(i.e Quetta Gladiators) won most of the matches 

# Visualizing the toss wins and match wins

In [14]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('Team')
ax1.set_ylabel('Count of toss wins')
ax1.set_title("toss winners")
temp1.plot(kind='bar')

ax2 = fig.add_subplot(122)
temp2.plot(kind = 'bar')
ax2.set_xlabel('Team')
ax2.set_ylabel('count of matches won')
ax2.set_title("Match winners")

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Match winners')

##### Checking If there is any null value in our data

In [15]:
 df.apply(lambda x: sum(x.isnull()),axis=0) 
    #find the null values in every column

Team_1              0
Team_2              0
Team1_Code          0
Team2_Code          0
Season              0
venue               0
id                  0
Win_By_Wkts         0
Win_By_Runs         0
toss_winner         0
toss_winner_code    0
toss_decision       0
result              0
winner              0
winner_code         0
team1_total_wins    0
team2_total_wins    0
dtype: int64

In [16]:
df.shape

(127, 17)

# Building A predictive Model

K-Fold creates multiple random splits of the dataset into training and validation data.
For each such split, the model is fit to the training data,
and predictive accuracy is assessed using the validation data

In [17]:
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn import metrics

def classification_model(model, data, predictors, outcome):
    model.fit(data[predictors],data[outcome])
    predictions = model.predict(data[predictors])
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    kf = KFold(n_splits=7,random_state=0)
    error = []
    for train, test in kf.split(data[predictors]):
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print('Cross-Validation Score : %s' % '{0:.3%}'.format(np.mean(error)))

    model.fit(data[predictors],data[outcome]) 

###### To avoid any warnings 

In [18]:
import warnings
warnings.filterwarnings('ignore')

# RandomForestClassifier

In [19]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=60)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)

Accuracy : 79.528%
Cross-Validation Score : 48.747%


In [20]:
team1='IU'
team2='QG'
toss_winner='QG'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'2','27','30']
input

[4, 6, 6, '2', '27', '30']

In [21]:
temp1=df['toss_winner_code'].value_counts(sort=True)
temp2=df['winner_code'].value_counts(sort=True)
#'No of toss winners by each team'
print('No of toss winners by each team')
for idx, val in temp1.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print('No of match winners by each team')
for idx, val in temp2.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print("Where MA : Match Abandoned\n      MT: Match Tied")

No of toss winners by each team
QG -> 31
PZ -> 28
KK -> 26
IU -> 20
LQ -> 12
MS -> 10
No of match winners by each team
QG -> 30
PZ -> 28
IU -> 27
KK -> 18
MS -> 10
LQ -> 10
MA -> 2
MT -> 2
Where MA : Match Abandoned
      MT: Match Tied


field: 1 bat 2

# MAKING PREDICTIONS

In [22]:
team1='IU'
team2='QG'
toss_winner='QG'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'2','27','30']
input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

The Winner is QG


In [23]:
team1='QG'
team2='PZ'
toss_winner='QG'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'1','30','28']

input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

The Winner is QG


In [ ]:
team1='KK'
team2='LQ'
toss_winner='LQ'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'1','18','10']

input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

The Winner is KK


In [ ]:
team1='IU'
team2='KK'
toss_winner='KK'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'1','27','18']

input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

The Winner is KK


#### Important features of model

In [ ]:
imp_input = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(imp_input)

toss_winner_code    0.283646
Team2_Code          0.185054
Team1_Code          0.177452
team2_total_wins    0.170228
team1_total_wins    0.114462
toss_decision       0.069159
dtype: float64


# Features Plotting Functions

In [ ]:
def plot_feature_importances(model, feature_names):
    import numpy as np
    c_features = len(feature_names)
    plt.barh(range(c_features), model.feature_importances_)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature name")
    plt.yticks(np.arange(c_features), feature_names)

In [ ]:
imp_input = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(imp_input)
plt.figure(figsize=(7,4))
plt.title("Model's Feature Importance")
plot_feature_importances(model, predictor_var)
plt.tight_layout()
plt.show()

toss_winner_code    0.283646
Team2_Code          0.185054
Team1_Code          0.177452
team2_total_wins    0.170228
team1_total_wins    0.114462
toss_decision       0.069159
dtype: float64


<IPython.core.display.Javascript object>

# DATA

In [ ]:
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# K Nearest Neighbor


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(X_train, y_train)
print('Psl dataset: KNN')
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of KNN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Psl dataset: KNN
Accuracy of KNN classifier on training set: 0.77
Accuracy of KNN classifier on test set: 0.47


In [ ]:
k_range = range(1,20)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

plt.figure()
plt.title("K value VS accuracy")
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20]);

<IPython.core.display.Javascript object>

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

knn = KNeighborsClassifier(n_neighbors = 1)

plt.figure()

for s in t:

    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        knn.fit(X_train, y_train)
        scores.append(knn.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
plt.title("Training set proportion VS Accuracy")
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

<IPython.core.display.Javascript object>

# Using Decision Tree

In [ ]:
#Data
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=0,max_depth=5, max_leaf_nodes=5,min_samples_leaf=5)
DT.fit(X_train, y_train)
print('Psl dataset: DT')
print('Accuracy of DT classifier on training set: {:.2f}'
     .format(DT.score(X_train, y_train)))
print('Accuracy of DT classifier on test set: {:.2f}'
     .format(DT.score(X_test, y_test)))

Psl dataset: DT
Accuracy of DT classifier on training set: 0.61
Accuracy of DT classifier on test set: 0.44


In [ ]:
leaf_nodes_range = range(2,6)
scores = []

for x in leaf_nodes_range:
    DT = DecisionTreeClassifier(random_state=0,max_depth=5, max_leaf_nodes=x,min_samples_leaf=5)
    DT.fit(X_train, y_train)
    scores.append(DT.score(X_test, y_test))

plt.figure()
plt.title("Leaf Nodes VS Accuracy")
plt.xlabel('nodes')
plt.ylabel('accuracy')
plt.scatter(leaf_nodes_range, scores)
plt.xticks([1,2,3,4,5]);

<IPython.core.display.Javascript object>

In [ ]:
depth_range = range(1,6)
scores = []

for x in depth_range:
    DT = DecisionTreeClassifier(random_state=0,max_depth=x, max_leaf_nodes=13,min_samples_leaf=5)
    DT.fit(X_train, y_train)
    scores.append(DT.score(X_test, y_test))

plt.figure()
plt.title("Depth VS Accuracy")
plt.xlabel('depth')
plt.ylabel('accuracy')
plt.scatter(depth_range, scores)
plt.xticks([0,1,2,3,4,5]);

<IPython.core.display.Javascript object>

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]
plt.figure()
for s in t:
    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        DT.fit(X_train, y_train)
        scores.append(DT.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
plt.title("Training set proportion VS Accuracy")
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

<IPython.core.display.Javascript object>

# Using Random Forest

In [ ]:
#Data
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0,max_depth=4)
RF.fit(X_train, y_train)
print('Psl dataset:RF')
print('Accuracy of RF classifier on training set: {:.2f}'
     .format(RF.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
     .format(RF.score(X_test, y_test)))

Psl dataset:RF
Accuracy of RF classifier on training set: 0.66
Accuracy of RF classifier on test set: 0.47


In [ ]:
depth_range = range(1,6)
scores = []

for z in depth_range:
    RF = RandomForestClassifier(random_state=0,max_depth=z)
    RF.fit(X_train, y_train)
    scores.append(RF.score(X_test, y_test))

plt.figure()
plt.title("Depth VS Accuracy")
plt.xlabel('depth')
plt.ylabel('accuracy')
plt.scatter(depth_range, scores)
plt.xticks([0,1,2,3,4,5]);

<IPython.core.display.Javascript object>

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]
plt.figure()
for s in t:
    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        DT.fit(X_train, y_train)
        scores.append(DT.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
plt.title("Training set proportion VS Accuracy")
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

<IPython.core.display.Javascript object>

# Using KFold Validation

In [ ]:
#Data
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn import metrics

def classification_model(model, data, predictors, outcome):
    model.fit(data[predictors],data[outcome])
    predictions = model.predict(data[predictors])
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    kf = KFold(n_splits=7,random_state=None)
    error = []
    for train, test in kf.split(data[predictors]):
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print('Cross-Validation Score : %s' % '{0:.3%}'.format(np.mean(error)))

    model.fit(data[predictors],data[outcome]) 

# KNN with cross validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 1)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)

# Decision Tree with cross Validation

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)

# Random Forest with Cross Validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=60)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)